In [ ]:
#!pip install googletrans==4.0.0-rc1
#!pip install gTTS

In [ ]:
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from googletrans import Translator
from gtts import gTTS
import os
import json
from collections import Counter
from IPython.display import Audio

# Step 1: Fetch News from Yahoo
def get_yahoo_news(company, max_articles=10):
    """Fetches news articles from Yahoo News for a given company."""
    search_url = f'https://news.search.yahoo.com/search?p={company}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(search_url, headers=headers)

    if response.status_code != 200:
        return []  # Avoid crashing

    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []

    for item in soup.find_all('div', class_='NewsArticle', limit=max_articles):
        title_tag = item.find('h4')
        summary_tag = item.find('p')

        if title_tag and summary_tag:
            articles.append({
                'Title': title_tag.text.strip(),
                'Summary': summary_tag.text.strip(),
            })

    return articles

# Step 2: Perform Sentiment Analysis
def analyze_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    return "Positive" if polarity > 0.05 else "Negative" if polarity < -0.05 else "Neutral"

# Step 3: Comparative Sentiment Analysis
def comparative_sentiment_analysis(news_articles):
    sentiment_results = [analyze_sentiment(article["Summary"]) for article in news_articles]
    sentiment_counts = Counter(sentiment_results)

    return {
        "Sentiment Distribution": sentiment_counts,
        "Coverage Differences": [
            {
                "Comparison": "Some articles are positive while others discuss challenges.",
                "Impact": "Positive news boosts investor confidence, while negative news may cause concerns."
            }
        ]
    }

# Step 4: Translate Text to Hindi
def translate_to_hindi(text):
    translator = Translator()
    return translator.translate(text, src="en", dest="hi").text

# Step 5: Convert Hindi Text to Speech
def text_to_speech_hindi(text, filename="news_audio.mp3"):
    tts = gTTS(text=text, lang="hi")
    tts.save(filename)
    return filename

# 🚀 **Main Execution (Single Input)**
company_name = input("Enter the company name: ")
print("\nFetching News, Performing Sentiment Analysis, Translating to Hindi, and Generating Speech...\n")

news_articles = get_yahoo_news(company_name)

# Process each article (Sentiment Analysis + Translation)
hindi_summaries = []
for article in news_articles:
    article["Sentiment"] = analyze_sentiment(article["Summary"])
    hindi_summary = translate_to_hindi(article["Summary"])
    hindi_summaries.append(hindi_summary)

# Perform Comparative Sentiment Analysis
comparative_results = comparative_sentiment_analysis(news_articles)

# Convert all summaries into Hindi speech
speech_text = " ".join(hindi_summaries)
speech_file = text_to_speech_hindi(speech_text)

# Final JSON Output
final_output = {
    "Company": company_name,
    "Articles": news_articles,
    "Comparative Sentiment Score": comparative_results,
    "Final Sentiment Analysis": "Overall, the news coverage is mixed.",
    "Audio": speech_file
}

# Print JSON Output
print("\n✅ JSON Output:")
print(json.dumps(final_output, indent=4, ensure_ascii=False))

# Play Hindi Speech
print("\n🎙 Playing Hindi Speech...")
Audio(speech_file, autoplay=True)


Enter the company name: Tesla

Fetching News, Performing Sentiment Analysis, Translating to Hindi, and Generating Speech...


✅ JSON Output:
{
    "Company": "Tesla",
    "Articles": [
        {
            "Title": "Tesla owners dump cars amid mounting hate — but not all is lost",
            "Summary": "Tesla owners are finding themselves at a crossroads as vandalism of their vehicles and the company's...",
            "Sentiment": "Neutral"
        },
        {
            "Title": "Tesla insurance has always been pricey. Will vandalism make it higher?",
            "Summary": "Incidents of vandalism against Teslas have risen along with anger against CEO Elon Musk, whose...",
            "Sentiment": "Negative"
        },
        {
            "Title": "Tesla: Sales and EPS Growth Are Slowing - What’s Next for the Stock? |...",
            "Summary": "Tesla (NASDAQ:TSLA) is in the news for all the wrong reasons. Here is a very brief take from a...",
            "Sentiment": "Negativ